In [ ]:
#real time:
import tensorflow as tf
from tensorflow import keras
import cv2
import mediapipe as mp
import numpy as np

def real_time_pose_classification():
    # Load the model
    model = keras.models.load_model('C:\\Users\\Santosh\\Desktop\\fyp\\src\\saved_models\\cnn_landmark_model.keras')

    # Define pose mapping
    pose_map = {0: 'downdog', 1: 'goddess', 2: 'plank', 3: 'tree', 4: 'warrior'}

    # Initialize MediaPipe Pose
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Open the video capture
    cap = cv2.VideoCapture(0)  # 0 for default webcam

    while True:
        # Read a frame from the video capture
        ret, frame = cap.read()

        # Convert the frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame and find the pose
        result = pose.process(frame_rgb)

        # Create a blank numpy array with zeros
        skeleton_image = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)

        # Draw the skeleton on the blank image
        if result.pose_landmarks:
            mp_drawing.draw_landmarks(
                skeleton_image,
                result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=5, circle_radius=5),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2)
            )

        # Resize the skeleton image to the required input size
        skeleton_image_resized = cv2.resize(skeleton_image, (75, 75))

        # Convert to float and normalize
        img_array = skeleton_image_resized.astype('float32') / 255.0

        # Expand dimensions for model input
        img_array = np.expand_dims(img_array, axis=0)

        # Make the prediction
        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction, axis=1)[0]
        predicted_pose_name = pose_map[predicted_class_index]

        # Display the predicted pose
        cv2.putText(frame, f"Predicted pose: {predicted_pose_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Pose Classification', frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close the window
    cap.release()
    cv2.destroyAllWindows()

# Call the real_time_pose_classification function
real_time_pose_classification()